In [1]:
import MDAnalysis as mda

/home/kalki/src/MD/polymer/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
uni = mda.Universe("nvt_vac_60ns_run.gro", "nvt_vac_60ns_run.xtc")

In [ ]:
for atom in uni.atoms:
    print(atom.id)

In [4]:
from MDAnalysis.core.groups import AtomGroup

In [11]:
dummy_grp = AtomGroup([1,2,3], uni) 

In [ ]:
com_coords = dummy_grp.center_of_mass()

In [13]:
com_coords

array([96.09218784, 49.73032913, 35.72079483])

In [14]:
cog_coords = dummy_grp.center_of_geometry()

In [15]:
cog_coords

array([96.14666239, 49.82333374, 35.66333135])

In [34]:
def get_cog_map_peg8pla8(uni, num_residue=10):
    master_cog_map = {}
    for i in range(num_residue):
        grp = [atom for atom in uni.atoms[i * 130:(i + 1) * 130]]
        PEGc = grp[:24]
        PLAc = grp[24:64]
        PEGc_indiv_split = [[atom for atom in PEGc[j * 3:(j + 1) * 3]]
                            for j in range(8)]
        PLAc_indiv_split = [[atom for atom in PLAc[k * 5:(k + 1) * 5]]
                            for k in range(8)]

        cog_map = {}

        for j, PEG_monomer in enumerate(PEGc_indiv_split):
            mono_ids = [mono.id for mono in PEG_monomer]
            assert len(mono_ids) == 3
            peg_cog = AtomGroup(mono_ids, uni).center_of_geometry()
            cog_map[f"PEG_{j}"] = peg_cog

        for k, PLA_monomer in enumerate(PLAc_indiv_split):
            assert len(PLA_monomer) == 5
            mono_ids = [mono.id for mono in PLA_monomer]
            pla_cog = AtomGroup(mono_ids, uni).center_of_geometry()
            cog_map[f"PLA_{k}"] = pla_cog

        master_cog_map[f"res_{i}"] = cog_map
    return master_cog_map

In [35]:
cog_map_peg8pla8 = get_cog_map_peg8pla8(uni)

In [36]:
cog_map_peg8pla8

{'res_0': {'PEG_0': array([96.14666239, 49.82333374, 35.66333135]),
  'PEG_1': array([94.13966624, 47.75666555, 37.8466657 ]),
  'PEG_2': array([92.61999766, 46.21000036, 40.37333043]),
  'PEG_3': array([92.68366241, 48.63999812, 42.44999822]),
  'PEG_4': array([92.14666494, 47.54999797, 44.88666534]),
  'PEG_5': array([89.15333049, 47.12333043, 46.40633011]),
  'PEG_6': array([87.33666484, 47.6566658 , 48.30666351]),
  'PEG_7': array([85.12333425, 46.49999873, 49.97999954]),
  'PLA_0': array([82.74199982, 47.30599899, 51.73199844]),
  'PLA_1': array([83.85379944, 48.48799973, 54.62399902]),
  'PLA_2': array([82.32199554, 50.6219986 , 56.95      ]),
  'PLA_3': array([79.50999603, 52.0961998 , 58.77999878]),
  'PLA_4': array([76.34199829, 52.52979889, 60.38219757]),
  'PLA_5': array([76.06399689, 49.05179901, 61.95599747]),
  'PLA_6': array([76.83999939, 46.78019791, 64.24799805]),
  'PLA_7': array([78.81759796, 47.20999985, 58.77619781])},
 'res_1': {'PEG_0': array([ 2.95666651, 61.613

In [37]:
def determine_angle_between_points(p1, p2, p3):
    import numpy as np

    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p2)

    cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    angle_rad = np.arccos(cos_theta)
    angle_deg = np.degrees(angle_rad)

    return angle_deg

In [44]:
def iterate_conj_3_points(cog_chain_map):
    seg_sequence = list(cog_chain_map.keys())
    for i in range(len(seg_sequence) - 2):
        seg1 = seg_sequence[i]
        seg2 = seg_sequence[i + 1]
        seg3 = seg_sequence[i + 2]

        point1 = cog_chain_map[seg1]
        point2 = cog_chain_map[seg2]
        point3 = cog_chain_map[seg3]

        yield (seg1, seg2, seg3, point1, point2, point3)


In [45]:
for points in iterate_conj_3_points(cog_map_peg8pla8["res_0"]):
    print("points >>", points)

points >> ('PEG_0', 'PEG_1', 'PEG_2', array([96.14666239, 49.82333374, 35.66333135]), array([94.13966624, 47.75666555, 37.8466657 ]), array([92.61999766, 46.21000036, 40.37333043]))
points >> ('PEG_1', 'PEG_2', 'PEG_3', array([94.13966624, 47.75666555, 37.8466657 ]), array([92.61999766, 46.21000036, 40.37333043]), array([92.68366241, 48.63999812, 42.44999822]))
points >> ('PEG_2', 'PEG_3', 'PEG_4', array([92.61999766, 46.21000036, 40.37333043]), array([92.68366241, 48.63999812, 42.44999822]), array([92.14666494, 47.54999797, 44.88666534]))
points >> ('PEG_3', 'PEG_4', 'PEG_5', array([92.68366241, 48.63999812, 42.44999822]), array([92.14666494, 47.54999797, 44.88666534]), array([89.15333049, 47.12333043, 46.40633011]))
points >> ('PEG_4', 'PEG_5', 'PEG_6', array([92.14666494, 47.54999797, 44.88666534]), array([89.15333049, 47.12333043, 46.40633011]), array([87.33666484, 47.6566658 , 48.30666351]))
points >> ('PEG_5', 'PEG_6', 'PEG_7', array([89.15333049, 47.12333043, 46.40633011]), arra

In [47]:
master_angle_repo = {}
for seg, cog_map in cog_map_peg8pla8.items():
    angle_repo = {}
    for seg1, seg2, seg3, p1, p2, p3 in iterate_conj_3_points(cog_map):
        angle = determine_angle_between_points(p1, p2, p3)
        angle_repo[f"{seg1}_{seg2}_{seg3}"] = angle
    master_angle_repo[seg] = angle_repo

In [48]:
print(master_angle_repo)

{'res_0': {'PEG_0_PEG_1_PEG_2': np.float64(167.79077187035912), 'PEG_1_PEG_2_PEG_3': np.float64(97.51345427364686), 'PEG_2_PEG_3_PEG_4': np.float64(105.8479743591815), 'PEG_3_PEG_4_PEG_5': np.float64(128.81456730512312), 'PEG_4_PEG_5_PEG_6': np.float64(153.1380772609527), 'PEG_5_PEG_6_PEG_7': np.float64(144.73118485516707), 'PEG_6_PEG_7_PLA_0': np.float64(142.11351317083842), 'PEG_7_PLA_0_PLA_1': np.float64(109.37964748227655), 'PLA_0_PLA_1_PLA_2': np.float64(130.4329299005809), 'PLA_1_PLA_2_PLA_3': np.float64(155.60496347653495), 'PLA_2_PLA_3_PLA_4': np.float64(162.20801513942973), 'PLA_3_PLA_4_PLA_5': np.float64(97.95348383216957), 'PLA_4_PLA_5_PLA_6': np.float64(152.73214361207795), 'PLA_5_PLA_6_PLA_7': np.float64(51.76694621546177)}, 'res_1': {'PEG_0_PEG_1_PEG_2': np.float64(167.8128130104114), 'PEG_1_PEG_2_PEG_3': np.float64(97.42858419132511), 'PEG_2_PEG_3_PEG_4': np.float64(105.76534796714235), 'PEG_3_PEG_4_PEG_5': np.float64(128.6681005357158), 'PEG_4_PEG_5_PEG_6': np.float64(1

In [51]:
def get_mean_angle_between_PEG(angle_repo):
    import numpy as np

    angles = []
    for key, angle in angle_repo.items():
        if key.startswith("PEG_") and "PLA" not in key:
            angles.append(angle)

    mean_angle = np.mean(angles) if angles else None
    return mean_angle

In [54]:
master_angles = []
for seq in master_angle_repo.values():
    mean_peg_angle = get_mean_angle_between_PEG(seq)
    master_angles.append(mean_peg_angle)

In [56]:
import numpy as np

In [57]:
np.mean(master_angles)

np.float64(125.42354170279312)

## The mean angle between PEG blocks itself in the chain was found to be `125.42 D` , which is approximate to `123 D` mentioned in the martini3 force field file for PEO. 

In [59]:
master_peg_peg_pla = []
master_peg_pla_pla = []
for seq, vals in master_angle_repo.items():
    master_peg_peg_pla.append(vals["PEG_6_PEG_7_PLA_0"])
    master_peg_pla_pla.append(vals["PEG_7_PLA_0_PLA_1"])
np.mean(master_peg_peg_pla), np.mean(master_peg_pla_pla)

(np.float64(137.78764297022985), np.float64(104.23980618885582))

## The mean angle for case of PEG-PEG-PLA is `137.78 D`.
## The mean angle for case of PEG-PLA-PLA is `104.239 D`. 

In [60]:
def get_mean_angle_between_PLA(angle_repo):
    import numpy as np

    angles = []
    for key, angle in angle_repo.items():
        if key.startswith("PLA_") and "PEG" not in key:
            angles.append(angle)

    mean_angle = np.mean(angles) if angles else None
    return mean_angle

In [61]:
master_pla_angles = []
for seq in master_angle_repo.values():
    mean_pla_angle = get_mean_angle_between_PLA(seq)
    master_pla_angles.append(mean_pla_angle)

In [ ]:
mean_pla_angle = np.mean(master_pla_angles)
mean_pla_angle

np.float64(120.60039753360738)

## The mean angle between PLA-PLA-PLA block was found to be `120.6 D`.

Analyzing normalized bond lengths between PLA-PLA blocks. 

In [ ]:
for residue in uni.residues:
    req_atoms = residue.atoms[24:64]
    for i,a in enumerate(req_atoms):
        print(i, a)
    for a,b in zip(req_atoms[3::5], req_atoms[5::5]):
        print(a,b) 
    break

0 <Atom 25: O8 of type O of resname LIG, resid 1 and segid SYSTEM>
1 <Atom 26: C16 of type C of resname LIG, resid 1 and segid SYSTEM>
2 <Atom 27: C17 of type C of resname LIG, resid 1 and segid SYSTEM>
3 <Atom 28: C18 of type C of resname LIG, resid 1 and segid SYSTEM>
4 <Atom 29: O9 of type O of resname LIG, resid 1 and segid SYSTEM>
5 <Atom 30: O10 of type O of resname LIG, resid 1 and segid SYSTEM>
6 <Atom 31: C19 of type C of resname LIG, resid 1 and segid SYSTEM>
7 <Atom 32: C20 of type C of resname LIG, resid 1 and segid SYSTEM>
8 <Atom 33: C21 of type C of resname LIG, resid 1 and segid SYSTEM>
9 <Atom 34: O11 of type O of resname LIG, resid 1 and segid SYSTEM>
10 <Atom 35: O12 of type O of resname LIG, resid 1 and segid SYSTEM>
11 <Atom 36: C22 of type C of resname LIG, resid 1 and segid SYSTEM>
12 <Atom 37: C23 of type C of resname LIG, resid 1 and segid SYSTEM>
13 <Atom 38: C24 of type C of resname LIG, resid 1 and segid SYSTEM>
14 <Atom 39: O13 of type O of resname LIG, res